<a href="https://colab.research.google.com/github/Louisfhgr/Dash.py/blob/master/SUGO_RNN_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Projekt SUGO RNN


### Datenfluss

#### Datenfluss automatisiert

Datenfluss manuell (csv)

#### Wetterdaten

### Explorative Datenanalyse

#### SUGO Artikel (SAP)

#### Wetterdaten

## KI Modelle

## Modell Evaluation